<a href="https://colab.research.google.com/github/PurpleDin0/698S_BOG/blob/master/698S_Project_BOG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Web Scraper Execution Notebook
BLUF: THis notebook imports the relevant python scraping scripts from github, builds the envirnoment, executes the scripts, saves the data locally, performs basic NLP on the recieved data, then prompts the user to save the resulting information/data (either locally, or to Google Drive).

* [ ] The websites to be searched are located here [Link to website list](https://drive.google.com/file/d/1y8FL1rBu8yb2HEn7-ok-ILy1Wq3UrDZU/view?usp=sharing).  Note: website list restricted, for access see the author.
* [ ] The scrapped data should be searched for the terms included in this file [Link to search terms list](https://drive.google.com/file/d/18MyoEiAYTG6w6JRejv5d4qVK4RxYyRu2/view?usp=sharing).  Note: file access restricted, for access see the author.

Problem Statement:
Builds a tool to scrape websites and then analyze the information for key terms
and provide the analyst with a rank-ordered list of documents that require human review. 
1. [ ] Construct a set of Python scripts that can be used to scrape a set of [3 websites](https://drive.google.com/file/d/1y8FL1rBu8yb2HEn7-ok-ILy1Wq3UrDZU/view?usp=sharing).  
  - [ ] 1.1. Build script to scrape website 1.  
  - [ ] 1.2. Build script to scrape website 2.  
  - [ ] 1.3. Build script to scrape website 3.  
2. [ ] Construct a python Notebook that executes the python script and then performs natural language processing (NLP) to the scrapped data.  
  - [ ] 2.1. Build the initial section that executes the python scripts.
  - [ ] 2.2. Rank order the scrapped results by relevance to the analyist. The ranking will be based on the [list of key terms](https://drive.google.com/file/d/18MyoEiAYTG6w6JRejv5d4qVK4RxYyRu2/view?usp=sharing).
  - [ ] 2.3. include an ability to automate the translation of selected
portions of the scraped content.
  - [ ] 2.4. Provide the user with the ability to save the scrapped content.

- Other possible things: 
  -  The code should allow the list of keywords to change over time.  
  -  The system  should immediately flag certain keywords for human review regardless of other content in the document.  
  -  The system should be able to retrieve and analyze office documents (PDFs,
spreadsheets, and word processor documents). 
  - The system should perform sentiment analysis of selected content.


In [ ]:
!cp /content/698S_BOG/websites.csv '/content/drive/My Drive/Colab Notebooks/Coursework/698S/Project'

# Prepare the environment 
0. git the repo and install all dependancies
1. Install any dependancies
2. Load the list of search sites and search terms

In [1]:
import os
## 0. Clone the repo and then change the working directory to the cloned folder
git_repo =  '698S_BOG'  # Repo we want to clone
git_user = 'PurpleDin0' # User/org we want to clone the repo from
!git clone https://github.com/{git_user}/{git_repo}.git
repo_dir = os.path.join(os.getcwd(), git_repo)
os.chdir(repo_dir)

## 1. Install the required libraries
!pip install -r requirements.txt

# Caution: Once complete make sure to restart the runtime if required. 

Cloning into '698S_BOG'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 69 (delta 35), reused 18 (delta 5), pack-reused 0
Unpacking objects: 100% (69/69), done.
     |████████████████████████████████| 235kB 9.3MB/s 
     |████████████████████████████████| 102kB 11.0MB/s 
     |████████████████████████████████| 61kB 7.8MB/s 
     |████████████████████████████████| 61kB 8.6MB/s 
     |████████████████████████████████| 61kB 8.3MB/s 
     |████████████████████████████████| 3.1MB 31.3MB/s 
     |████████████████████████████████| 61kB 9.0MB/s 
     |████████████████████████████████| 2.7MB 72.3MB/s 
     |████████████████████████████████| 61kB 9.3MB/s 
     |████████████████████████████████| 235kB 64.8MB/s 
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.5-cp36-none-any.whl size=11515 sha256=26ebe20ee9d6492aefa420a9353ff6e9c94fc0a792db57bd67768ba2c185ee8d
  Stored in directory: /

In [3]:
## 2. Load the list of search sites and search terms

# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd


# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
# 
# Get the website list file and load it to a dataframe
file_id = '1y8FL1rBu8yb2HEn7-ok-ILy1Wq3UrDZU'  ## USER UPDATEABLE VALUE! ##
downloaded = drive.CreateFile({'id': file_id})
downloaded.FetchMetadata()
filename = downloaded.metadata['originalFilename']
downloaded.GetContentFile(filename)
website_list_df = pd.read_csv(filename)
# Get the search term file and load it to a dataframe
file_id = '18MyoEiAYTG6w6JRejv5d4qVK4RxYyRu2'
downloaded = drive.CreateFile({'id': file_id})
downloaded.FetchMetadata()
filename = downloaded.metadata['originalFilename'] ## USER UPDATEABLE VALUE! ##
downloaded.GetContentFile(filename)
search_terms_df = pd.read_csv(filename)



In [4]:
domain = [s.replace('www.', '') for s in website_list_df.Website.to_list()]
website_list_df['Domain'] = domain

In [6]:
# Cool banner printer
def banner(t, s='~'):
    l = s * (len(t) + 4)
    print(l + '\n' + '{0} {1} {0}'.format(s, t) + '\n' + l)

# Formatted line
def line(t):
  print("\n", "~"*(26-(len(t)//2)), t, "~"*(26-(len(t)//2)))

#1. Modify the python scripts to target them at the desired websites
Todolist:
 * [ ] edit the python to change the url to a passed variable the targeted site(s) with the desired targets
 * [ ]  

In [ ]:
#TODO write a script that removes the generic sites from the imported python code with the desired target(s)
#TODO 

#2. Run the spider script

In [5]:
ALLOWED_DOMAINS = [s.replace('www.', '') for s in website_list_df.Website.to_list()]
URLS = website_list_df.Website.to_list()
print(URLS)
print(ALLOWED_DOMAINS)

['www.miit.gov.cn', 'www.sastind.gov.cn', 'www.jmjh.miit.gov.cn']
['miit.gov.cn', 'sastind.gov.cn', 'jmjh.miit.gov.cn']


In [26]:
!python3 sixnineeight.py ALLOWED_DOMAINS='miit.gov.cn' URLS='http://www.miit.gov.cn'

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~ Running the Spider for links ~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
It worked!!
Output json file written to links.json.


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
             End : Mon Jun 22 00:49:59 2020
        Run Time : 00:03:06
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


## Load the script into memory


In [ ]:
# -*- coding: utf-8 -*-
"""
    Following PEP8 Style Guide and Google format function docstrings
    Website spider to extract list of URLs and saves to file.
    root/
    |_____file.json
"""

# Standard library imports one per line
import sys
from urllib.parse import urlparse, urljoin

# Third party library imports one per line
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.linkextractors.lxmlhtml import LxmlLinkExtractor
from scrapy.spiders import Rule

# Authorship constants
__authors__ = ['BA', 'JP']
__copyright__ = "Copyright 2020, NIU"
__credits__ = [None]
__version__ = "0.1"
__status__ = "Alpha"
__date__ = "10 March 2020"

# Global variables
visited = list()
NAME = 'sixnineeight'
ALLOWED_DOMAINS = '' # Base URL goes here  ## USER UPDATEABLE VALUE! ##
URLS = '' # Fully qualified domain goes here  ## USER UPDATEABLE VALUE! ##


class SixnineeightSpider(scrapy.Spider):
    """
        Summary:
            `SixnineeightSpider` instantiates the custom spider object.
            Parameters:
                scrapy.Spider (obj):
            Returns:
                (None)
    """
    name = NAME
    allowed_domains = [ALLOWED_DOMAINS]
    start_urls = [URLS]
    custom_settings = {
        'DEPTH_LIMIT': 1  # Gets deep quick...2 is very deep on large sites e.g. 45K+ unique links on CNN. 0=no limit
    }
    rules = (
        Rule(
            LxmlLinkExtractor(allow=()),
            callback='parse_obj',
            follow=True),
    )

    def parse(self, response):
        """
            Summary:
                `parse` method to parse spider responses.
                Parameters:
                    self (obj): self
                    response (obj): scrapy.Response
                Returns:
                    (None)
        """
        links = response.xpath('//a//@href').extract()
        for link in links:
            if "#" in link:
                link = link.split("#")[0]
            if link in visited or urljoin(URLS,
                                          link) in visited or "mailto:" in link or "tel:" in link or "javascript:" in link:
                continue
            else:
                if ALLOWED_DOMAINS in link and link in urlparse(link).netloc:
                    if urlparse(link).scheme == '':
                        link = urljoin(urlparse(URLS).scheme, urlparse(URLS).path, link)
                    visited.append(link)
                    yield {
                        "link": link
                    }
                    continue
                if urlparse(link).netloc == '' and link not in visited:
                    visited.append(urljoin(URLS, link))
                    yield {
                        "link": urljoin(URLS, link)
                    }
                    continue
                if urlparse(link).netloc != '' and ALLOWED_DOMAINS not in link:
                    continue

        for next_page in links:
            if next_page is not None and urlparse(next_page).netloc == '' or ALLOWED_DOMAINS in next_page:
                if urlparse(next_page).netloc == '':
                    next_page = urljoin(URLS, next_page)
                next_page = response.urljoin(next_page)
                yield scrapy.Request(next_page, callback=self.parse, dont_filter=True)

        pass

In [ ]:
# Prints a banner
from time import time, ctime, strftime, gmtime

def elapsed(start_time):
  """
    Calcuilates the elapsed time of script start of exeution until
    code completes. This allows for baselining different runtime enviroments
  """
  current_time = time()
  elapsed_time = current_time - start_time
  print("\n\n" + "~"*50)
  print("             End : " + ctime(current_time))
  print("        Run Time : " + strftime("%H:%M:%S", gmtime(elapsed_time)))
  print("~"*50)

start_time = time()

banner('Running the Spider for links')

# Generic spider object with custom settings
process = CrawlerProcess(
            settings={
              # 'FEED_FORMAT': 'pickle',
              # 'FEED_URI': 'file:///***/links.pkl',
              # 'LOG_LEVEL': 'INFO',  # Uncomment if you don't want scrapy to puke DEBUG to the console
              # 'DOWNLOAD_DELAY': 0.25,   # 250 ms of delay, default is random between 0.5 - 1.5
              'TELNETCONSOLE_ENABLED': False,  # On by default...that's dumb ¯\_(ツ)_/¯
              'FEED_FORMAT': 'json',
              'FEED_URI': 'links.json',
              'LOG_LEVEL': 'CRITICAL',
              'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36',
              'CLOSESPIDER_TIMEOUT': 60
            } 
          )

# Instantiates the custom spider class object
process.crawl(SixnineeightSpider)
try:
    # Kick off the custom spider and crawl
    process.start()

    print("It worked!!")
    print("Output {} file written to {}.".format(process.settings['FEED_FORMAT'], process.settings['FEED_URI']))
    elapsed(start_time)
except Exception as e:
    print("Well, that didn't work... \n {}".format(e))

## Execute the script that gets the links

- [ ] run this to execute the 

# Download the files and create a zip file for processing

In [ ]:
import os
import requests
import zipfile
import pandas as pd
from urllib.parse import urlparse, urljoin
from tqdm import tqdm

# Prints a banner
banner('Beginning file downloads')

# Sets custom user-agent string to masquerade 
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36'}

# Reads the links fro the json file and sorts for .pdf (or other) filetypes found
pdf_links = pd.read_json('/content/698S_BOG/links.json')
pdf_links = pdf_links[pdf_links['link'].str.contains(".pdf", na=False)] 

# Loops through the list of links
for url in pdf_links.link.values.tolist():
  # Gets the basename of the file and if a dynamic link sets a new name
  filename = os.path.basename(urlparse(url).path)
  if os.path.exists(os.path.join('content', filename)):
    filename = filename + urlparse(url).query + '.pdf'
  
  print('*'*50)
  print("Requesting: " + url)
  try:
    # Checks if content directory exists, if not, create
    if not os.path.exists('content'):
      os.makedirs('content')

    # Makes the request
    r = requests.get(url, stream=True, headers=headers)

    # Processes if link is valid, or raise generic exception
    if r.status_code == 200:
      # Chunks the file for progress bar
      total_size = int(r.headers.get('content-length', 0))
      block_size = 1024 

      # Progress bar parameters
      t=tqdm(total=total_size, unit='iB', unit_scale=True)
      
      # Downloads the files to the content folder
      with open(os.path.join('content', filename), 'wb') as f:
          for data in r.iter_content(block_size):
              t.update(len(data))
              f.write(data)
      
      # Error handling
      if total_size != 0 and t.n != total_size:
        raise
      
      # Close the progress bar object
      t.close()
    else:
      print('{} Response, Skipping {}.'.format(r.status_code, url))
    
  except Exception as e:
    print("Error: {}".format(e))

# Zip up the files
banner("Zipping up the files")

# Zip object walks directory and packs all files without directory structure
try:
  ziper = zipfile.ZipFile('content.zip', 'w', zipfile.ZIP_DEFLATED)
  for root, dirs, files in os.walk('content/'):
    for file in files:
      ziper.write(os.path.join(root, file), file)
  ziper.close()
except Exception as e:
  print("Could not zip the file. " + e)

# Cleans up the files and removes the content directory
try:
  if os.path.exists('content'):
    for root, dirs, files in os.walk('content/'):
      for file in tqdm(files):
        os.remove(os.path.join(root, file))
    os.removedirs('content')
  if os.path.exists('content.zip'):
    print('\nZip file saved')
  else:
    raise Exception("Could not write zip file.")
except Exception as e:
  print(e)

In [ ]:
#Option one for calling the file
#pass the list of sites you want to scrape
# %cd {repo_dir}
# import sixnineeight

In [ ]:
#Option two for calling the file
# %cd /content/698S_BOG
# !python3 sixnineeight.py

# Parse the documents for the desired text


## Import parsing libraries

In [ ]:
!pip install tika

In [ ]:
# Import packages needed form parsing
import os
import pandas as pd
import tika
tika.initVM()
from tika import parser

## Load the search times

In [ ]:
# My take on unzipping the files again
import zipfile

working_folder = os.path.join('/content', '698S_BOG')  ## DEBUG ## hard_coded
filename = os.path.join(working_folder, 'content.zip')
scrape_folder = os.path.join(working_folder, 'scraped_files')

try:
  if not os.path.exists(scrape_folder):
    os.makedirs(scrape_folder)

  with zipfile.ZipFile(filename, 'r') as z:
    z.extractall(scrape_folder)
    
  os.chdir(scrape_folder)
  line("Files unzipped")
except Exception as e:
  print(e)



In [ ]:
# Get the search term file and load it to a dataframe
file_id = '18MyoEiAYTG6w6JRejv5d4qVK4RxYyRu2'
downloaded = drive.CreateFile({'id': file_id})
downloaded.FetchMetadata()
filename = downloaded.metadata['originalFilename']
downloaded.GetContentFile(filename)
search_terms_df = pd.read_csv(filename)

os.chdir(scrape_folder)

## Open the PDFs and parse the contents

In [ ]:
# open all the PDFs and parse their content into dataframes
pdf_df = pd.DataFrame()
for filename in os.listdir(scrape_folder):
    if filename.endswith(".pdf"):
        parsed = parser.from_file(filename)
        row = pd.DataFrame(
            [[filename, scrape_folder, parsed["metadata"], parsed["content"]]],
            index = [filename],
            columns = ['filename', 'folder', 'metadata', 'content'])
        pdf_df = pdf_df.append(row)
for term in search_terms_df['Term (CN)']:
    pdf_df[term] = pdf_df['content'].apply(lambda x : x.count(term))


################## future CODE ###############
#def content_score([occurance_list]):
#  for term in search_terms_df['Term (CN)']:
    
#for 
#pdf_df['sum'] = pdf_df[search_terms_df['Term (CN)']].apply(lambda x : x)
#pdf_df

### Show the dataframe

In [ ]:
pdf_df

# ===================================
# !!EVERYTHING WORKES TO THIS POINT!!

Everything down here is trash (and @PurpleDin0's)

## OLD CODE

In [ ]:
# Get the zip file and unzip
# file_id = '1wbud3KFgJ8Su6ChrM87BshB01F7uOm3M'
# downloaded = drive.CreateFile({'id': file_id})
# filename = 'content.zip' # changed from downloaded.metadata['originalFilename']
# downloaded.GetContentFile(filename)
# search_terms_df = pd.read_csv('search_terms.csv')
# scrape_folder = 'scraped_files'
# os.mkdir = scrape_folder

# !unzip {filename} -d {scrape_folder}
# os.chdir(scrape_folder)

In [ ]:
# open all the files
for file in os.listdir(scrape_folder):
    if file.endswith(".pdf"):
        print(file)


In [ ]:
# filename = '7980416.pdf'

In [ ]:
search_terms_df

In [ ]:
search_terms_df['Term (CN)'].to_list()

In [ ]:
import re
import tika
from tika import parser

# Kick off the virtual machine
tika.initVM()

# Loops through the files
banner('Searching the files for terms')

for file in os.listdir(scrape_folder):
    if file.endswith(".pdf"):
      parsed = parser.from_file(file)
      # Looks for the terms one by one in each file
      for term in search_terms_df['Term (CN)'].to_list():
        # Action if term found
        if term in parsed["content"]:
          count = [t.start() for t in re.finditer(term, parsed['content'])] # This is actially a list of the occurence locations, may be useful later ¯\_(ツ)_/¯
          line("Found {} occurences of term {} in file {}".format(len(count), term, file))

#print(parsed["metadata"])
#print(parsed["content"])

## Parse the documents for the desired text


## Import parsing libraries

In [ ]:
!pip install tika


In [ ]:
# Import packages needed form parsing
import os
import pandas as pdimport tika
tika.initVM()
from tika import parser

## Load the search terms

In [ ]:
# Get the search term file and load it to a dataframe
file_id = '18MyoEiAYTG6w6JRejv5d4qVK4RxYyRu2'
downloaded = drive.CreateFile({'id': file_id})
downloaded.FetchMetadata()
filename = downloaded.metadata['originalFilename']
downloaded.GetContentFile(filename)
search_terms_df = pd.read_csv(filename)

os.chdir(scrape_folder)